In [1]:
import sys
import collections
import mxnet as mx
from mxnet import autograd, gluon, init, metric, nd
from mxnet.gluon import loss as gloss, nn, rnn
from mxnet.contrib import text
import os
import random
import zipfile
from sklearn.model_selection import train_test_split
import spacy
import time
from time import strftime
from gensim import corpora
from gensim import models
from spacy.lemmatizer import Lemmatizer

D:\Anaconda3\envs\gluon\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
D:\Anaconda3\envs\gluon\lib\site-packages\scipy\sparse\sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
ROOT_PATH  = "data/"

In [3]:
def noation_replace(line):
    notations = ['?', '¿', ',', '.', '¡','!', ':', ",", ';', '-', '']
    output = line.strip()
    for i in notations:
        output = output.replace(i, '')
    return output.split('\t')

In [4]:
es_e_l = []
es_s_l = []
es_e_r = []
es_s_r = []
es_labels = []
## english-spanish text 
with open(os.path.join(ROOT_PATH, "cikm_english_train_20180516.txt"), 'r', encoding='utf-8') as esf:
    for line in esf:
        segs = noation_replace(line)
        es_e_l.append(segs[0].lower())
        es_e_r.append(segs[2].lower())
        es_s_l.append(segs[1].lower())
        es_s_r.append(segs[3].lower())
        es_labels.append(int(segs[4]))
        
se_e_l = []
se_s_l = []
se_e_r = []
se_s_r = []
se_labels = []
## spanish-english text
with open(os.path.join(ROOT_PATH, "cikm_spanish_train_20180516.txt"), 'r', encoding='utf-8') as ssf:
    for line in ssf:
        segs = noation_replace(line)
        se_s_l.append(segs[0].lower())
        se_s_r.append(segs[2].lower())
        se_e_l.append(segs[1].lower())
        se_e_r.append(segs[3].lower())
        se_labels.append(int(segs[4]))

test_s_1 = []
test_s_2 = []
## spanish test file
with open(os.path.join(ROOT_PATH, "cikm_test_a_20180516.txt"), 'r', encoding='utf-8') as tef:
    for line in tef:
        segs = noation_replace(line)
        test_s_1.append(segs[0].lower())
        test_s_2.append(segs[1].lower())

print("es data size:", len(es_s_l))
print("se data size:", len(se_e_l))
print("test data size:", len(test_s_1))

es data size: 20000
se data size: 1400
test data size: 5000


In [5]:
##add data sets (s0, s1, y) + (s1, s0, y)
left_texts = es_s_l + se_s_l
right_texts = es_s_r + se_s_r
y = es_labels + se_labels

print("left data size:", len(left_texts))
print("right data size:", len(right_texts))
print("label size:", len(y))

left data size: 21400
right data size: 21400
label size: 21400


In [6]:
span_sw = []
with open(os.path.join(ROOT_PATH, "spanish.txt"), 'r', encoding='utf-8') as swf:
    for line in swf:
        word = line.strip()
        span_sw.append(word)

In [7]:
spacy_en = spacy.load('en')
spacy_es = spacy.load('es')

def en_tokenizer(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

def es_tokenizer(text):
    return [tok.text for tok in spacy_es.tokenizer(text) if tok.text not in span_sw]

D:\Anaconda3\envs\gluon\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
D:\Anaconda3\envs\gluon\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]


In [8]:
left_tokenized = []
right_tokenized = []
test_left_tokenized = []
test_right_tokenized = []

for token in left_texts:
    left_tokenized.append(es_tokenizer(token))

for token in right_texts:
    right_tokenized.append(es_tokenizer(token))
    
for token in test_s_1:
    test_left_tokenized.append(es_tokenizer(token))

for token in test_s_2:
    test_right_tokenized.append(es_tokenizer(token))
    
    
token_counter = collections.Counter()
for sample in left_tokenized:
    for token in sample:
        if token not in token_counter:
            token_counter[token] = 1
        else:
            token_counter[token] += 1
            
for sample in right_tokenized:
    for token in sample:
        if token not in token_counter:
            token_counter[token] = 1
        else:
            token_counter[token] += 1
            
for sample in test_left_tokenized:
    for token in sample:
        if token not in token_counter:
            token_counter[token] = 1
        else:
            token_counter[token] += 1

for sample in test_right_tokenized:
    for token in sample:
        if token not in token_counter:
            token_counter[token] = 1
        else:
            token_counter[token] += 1
            
print("unique token count :", len(token_counter))

unique token count : 5824


In [9]:
# 根据词典，将数据转换成特征向量。
def encode_samples(tokenized_samples, vocab):
    features = []
    for sample in tokenized_samples:
        feature = []
        for token in sample:
            if token in vocab.token_to_idx:
                feature.append(vocab.token_to_idx[token])
            else:
                feature.append(0)
        features.append(feature)         
    return features

def pad_samples(features, maxlen=500, padding=0):
    padded_features = []
    for feature in features:
        if len(feature) > maxlen:
            padded_feature = feature[:maxlen]
        else:
            padded_feature = feature
            # 添加 PAD 符号使每个序列等长（长度为 maxlen ）。
            while len(padded_feature) < maxlen:
                padded_feature.append(padding)
        padded_features.append(padded_feature)
    return padded_features

In [10]:
vocab = text.vocab.Vocabulary(token_counter, unknown_token='<unk>', reserved_tokens=None)

left_texts_features = encode_samples(left_tokenized, vocab)
right_texts_featrues = encode_samples(right_tokenized, vocab)

left_padded_features = pad_samples(left_texts_features, 35, 0)
right_texts_featrues = pad_samples(right_texts_featrues, 35, 0)

test_left_features = encode_samples(test_left_tokenized, vocab)
test_right_featrues = encode_samples(test_right_tokenized, vocab)

test_left_padded_features = pad_samples(test_left_features, 35, 0)
test_right_texts_featrues = pad_samples(test_right_featrues, 35, 0)

In [11]:
num_validation_samples = int(0.2 * len(left_padded_features))

left_x_train = left_padded_features[:-num_validation_samples]
left_x_val = left_padded_features[-num_validation_samples:]

right_x_train = right_texts_featrues[:-num_validation_samples]
right_x_val = right_texts_featrues[-num_validation_samples:]

y_train = y[:-num_validation_samples]
y_val = y[-num_validation_samples:]

In [12]:
train_inputs = []
for i in range(len(left_x_train)):
    train_inputs.append([left_x_train[i], right_x_train[i]])
    
val_inputs = []
for i in range(len(left_x_val)):
    val_inputs.append([left_x_val[i], right_x_val[i]])

test_inputs = []
for i in range(len(test_left_padded_features)):
    test_inputs.append([test_left_padded_features[i], test_right_texts_featrues[i]])

In [13]:
ctx = mx.gpu()

train_features = nd.array(train_inputs, ctx=ctx)
train_label = nd.array(y_train, ctx)
val_features = nd.array(val_inputs, ctx=ctx)
val_label = nd.array(y_val, ctx)
test_featrues = nd.array(test_inputs, ctx=ctx)
# test_label = nd.array(test_label, ctx=ctx)

In [14]:
glove_embedding = text.embedding.CustomEmbedding(pretrained_file_path='data/wiki.es.vec', vocabulary=vocab)

D:\Anaconda3\envs\gluon\lib\site-packages\mxnet\contrib\text\embedding.py:278: UserWarning: At line 1 of the pre-trained text embedding file: token 985667 with 1-dimensional vector [300.0] is likely a header and is skipped.
  'skipped.' % (line_num, token, elems))


In [15]:
#相似度计算
def exponent_neg_manhattan_distance(left, right):
    output = nd.exp(-nd.sum(nd.abs(left - right), axis=1, keepdims=True))
#     print('the shape of left: ', left.shape)
#     print('the shape of right: ', right.shape)
#     print('the shape of output: ', output.shape)
    return output

def cos_distance(left, right):
    top = nd.sum(left * right, axis=1)
    normL = nd.sum(left ** 2, axis=1)
    normR = nd.sum(right ** 2, axis=1)
    output = top / ((normL ** 0.5) * (normR ** 0.5))
#     print('the shape of output: ', output.shape)
#     print(output)
    return output.reshape(100, 1)
    

# new metric
def logloss(y_true, y_pred):
    return -np.mean(y_true*np.log(y_pred) + (1-y_true)*np.log(1-y_pred))

In [16]:
def semilogy(x_vals, y_vals, x_label, y_label, x2_vals=None, y2_vals=None,
             legend=None, figsize=(3.5, 2.5)):
    gb.set_figsize(figsize)
    gb.plt.xlabel(x_label)
    gb.plt.ylabel(y_label)
    gb.plt.semilogy(x_vals, y_vals)
    if x2_vals and y2_vals:
        gb.plt.semilogy(x2_vals, y2_vals)
        gb.plt.legend(legend)
    gb.plt.show()

In [17]:
def nin_block(num_channels):
    blk = nn.Sequential()
    blk.add(nn.Conv2D(num_channels, kernel_size=1, activation='relu'),
            nn.Conv2D(num_channels, kernel_size=1, activation='relu'))
    return blk

In [24]:
class SentimentNet(nn.Block):
    def __init__(self, vocab, embed_size, num_hiddens, num_layers,
                 bidirectional, **kwargs):
        super(SentimentNet, self).__init__(**kwargs)
        with self.name_scope():
            self.embedding = nn.Embedding(len(vocab), embed_size)
                
#             self.encoder = rnn.LSTM(num_hiddens, num_layers=num_layers,
#                                     bidirectional=bidirectional,
#                                     input_size=embed_size)
            self.encoder = rnn.GRU(num_hiddens, num_layers, bidirectional=bidirectional, 
                                   input_size=embed_size, dropout=0.2)
            self.dense0 = nn.Dense(100, activation='relu', flatten=False)
#             self.dense0 = nin_block(100)
            self.dropout1 = nn.Dropout(0.5)
            self.norm2 = nn.BatchNorm()
#             self.dense1 = nin_block(50)
            self.dense1 = nn.Dense(50, activation='relu', flatten=False)
            self.dropout = nn.Dropout(0.5)
            self.norm3 = nn.BatchNorm()
            self.decoder = nn.Dense(num_outputs, flatten=False, activation='sigmoid')

    def forward(self, inputs):
        inputs = nd.transpose(inputs, axes=(1, 2, 0))
        
        bembeddings = self.embedding(inputs[0])
        bstates = self.encoder(bembeddings)
        # 连结初始时间步和最终时间步的隐藏状态。
        bencoding = nd.concat(bstates[0], bstates[-1])
        bencoding = self.dense0(bencoding)
        
        eembeddings = self.embedding(inputs[1])
        estates = self.encoder(eembeddings)
        # 连结初始时间步和最终时间步的隐藏状态。
        eencoding = nd.concat(estates[0], estates[-1])
        eencoding = self.dense0(eencoding)
        
        y = nd.subtract(bencoding, eencoding)
#         y = cos_distance(bencoding, eencoding)
        outputs = self.dropout1(y)
        outputs = self.norm2(outputs)
        outputs = self.dense1(outputs)
        outputs = self.dropout(outputs)
        outputs = self.norm3(outputs)
        outputs = self.decoder(outputs)
        
        return outputs

In [25]:
num_outputs = 1
lr = 0.1
num_epochs = 100
batch_size = 100
embed_size = 300
num_hiddens = 200
num_layers = 2
bidirectional = True

In [26]:
net = SentimentNet(vocab, embed_size, num_hiddens, num_layers, bidirectional)
net.initialize(init.Xavier(), ctx=ctx)
# 设置 embedding 层的 weight 为预训练的词向量。
net.embedding.weight.set_data(glove_embedding.idx_to_vec.as_in_context(ctx))
# 训练中不更新词向量（net.embedding中的模型参数）。
net.embedding.collect_params().setattr('grad_req', 'null')
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
loss = gloss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=True)

In [21]:
test_online_1 = []
test_online_2 = []
test_label = []
## spanish test file
with open(os.path.join(ROOT_PATH, "test_300_label.txt"), 'r', encoding='utf-8') as tef:
    for line in tef:
        segs = noation_replace(line)
        test_online_1.append(segs[0].lower())
        test_online_2.append(segs[1].lower())
        test_label.append(segs[2].lower())

test_online_left_tokenized = []
test_online_right_tokenized = []

for token in test_online_1:
    test_online_left_tokenized.append(es_tokenizer(token))

for token in test_online_2:
    test_online_right_tokenized.append(es_tokenizer(token))

test_online_left_features = encode_samples(test_online_left_tokenized, vocab)
test_online_right_featrues = encode_samples(test_online_right_tokenized, vocab)

test_online_left_padded_features = pad_samples(test_online_left_features, 40, 0)
test_online_right_texts_featrues = pad_samples(test_online_right_featrues, 40, 0)

test_online_inputs = []
for i in range(len(test_online_left_padded_features)):
    test_online_inputs.append([test_online_left_padded_features[i], test_online_right_texts_featrues[i]])

test_online_featrues = nd.array(test_online_inputs, ctx=ctx)
test_online_label= nd.array(test_label, ctx=ctx)

In [22]:
def eval_model(features, labels):
    l_sum = 0
    for i in range(features.shape[0] // batch_size):
        X = features[i*batch_size : (i+1)*batch_size].as_in_context(ctx)
        y = labels[i*batch_size :(i+1)*batch_size].as_in_context(ctx)
        output = net(X)
        l = loss(output, y)
        l_sum += l.sum().asscalar() 
    return l_sum / features.shape[0]

In [27]:
model_file = "model/Model_" + strftime("%Y-%m-%d %H-%M", time.localtime()) + ".mdl"
best_loss = 1.0

for epoch in range(1, num_epochs + 1):
    for i in range(train_features.shape[0] // batch_size):
        X = train_features[i * batch_size: (i + 1) * batch_size].as_in_context(ctx)
        y = train_label[i * batch_size: (i + 1) * batch_size].as_in_context(ctx)
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    train_loss = eval_model(train_features, train_label)
    val_loss = eval_model(val_features, val_label)
    test_loss = eval_model(test_online_featrues, test_online_label)
    if val_loss < best_loss :
        net.save_params(model_file)
        best_loss = val_loss
    print('epoch %d, train loss %.6f, val loss %.6f, test_loss %.6f' % (epoch, train_loss, val_loss, test_loss))

epoch 1, train loss 0.544655, val loss 0.511275, test_loss 0.652183
epoch 2, train loss 0.527403, val loss 0.498415, test_loss 0.631198
epoch 3, train loss 0.508451, val loss 0.486533, test_loss 0.604716
epoch 4, train loss 0.492175, val loss 0.475984, test_loss 0.583239
epoch 5, train loss 0.471631, val loss 0.462152, test_loss 0.569079
epoch 6, train loss 0.460753, val loss 0.455670, test_loss 0.548176
epoch 7, train loss 0.449730, val loss 0.449374, test_loss 0.548253
epoch 8, train loss 0.429281, val loss 0.437299, test_loss 0.544887
epoch 9, train loss 0.420902, val loss 0.440126, test_loss 0.544642
epoch 10, train loss 0.405040, val loss 0.427196, test_loss 0.544019
epoch 11, train loss 0.397702, val loss 0.428325, test_loss 0.538214
epoch 12, train loss 0.388908, val loss 0.424789, test_loss 0.533082
epoch 13, train loss 0.379741, val loss 0.421097, test_loss 0.535190
epoch 14, train loss 0.376968, val loss 0.419293, test_loss 0.530266
epoch 15, train loss 0.355802, val loss 0.4

In [ ]:
def predict(features):
    output = []
    for i in range(features.shape[0] // batch_size):
        X = features[i*batch_size : (i+1)*batch_size].as_in_context(ctx)
        y = net(X).asnumpy().flatten()
        for i in y:
            output.append(i)

    return output

In [ ]:
output = predict(test_featrues)

result_file_name = strftime("%Y-%m-%d %H-%M", time.localtime()) + "result.txt"

with open(result_file_name, 'w') as f:
    for line in output:
        f.write(str(round(line, 6)) + '\n')

In [ ]:
# TFIDF
texts = left_tokenized + right_tokenized + test_left_tokenized + test_right_tokenized
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts if text not in span_sw]

In [ ]:
tfidf = models.TfidfModel(corpus)
tfidf.save("model.tfidf")

In [ ]:
corpus_tfidf = tfidf[corpus]
output = []
for doc in corpus_tfidf:
    output.append(doc)

In [ ]:
for i in range(10):
    print(output[i])

In [ ]:
lda = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=5, iterations=500)

In [ ]:
lda.save('mylda_v1.pkl')

In [ ]:
for index, score in sorted(lda[corpus_tfidf[0]], key=lambda tup: -1*tup[1]):
    print("Score: %f\t Topic: %s \n" % (score, lda.print_topic(index, 5)))